In [18]:
import pandas as pd
import re
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
import nltk

In [19]:
def cluster_cohesion(embeddings, labels):
    scores = []
    for c in np.unique(labels):
        idx = labels == c
        if idx.sum() < 2:
            continue
        sims = cosine_similarity(embeddings[idx])
        scores.append(sims.mean())
    return float(np.mean(scores))

In [37]:
# data = pd.read_csv("output/songs_with_lyrics_cleaned.csv")
# separated_lyrics = data['lyrics'].dropna()

musical_data = pd.read_csv("dataset/musical.csv")
separated_lyrics = musical_data['Lyrics'].dropna()
# separated_lyrics = pd.concat([separated_lyrics, musical_data['Lyrics'].dropna()])

corpus = []

print("🧹 Cleaning and Segmenting Lyrics...")

for lyric_text in separated_lyrics:
    if isinstance(lyric_text, str):
        # 1. Clean Tags
        text_no_tags = re.sub(r'\[.*?\]', ' ', lyric_text, flags=re.DOTALL)
        
        # 2. Split into lines
        raw_segments = text_no_tags.split('\n')
        
        # 3. Clean and Filter
        clean_lines = [line.strip() for line in raw_segments if len(line.strip()) > 0]
        
        # 4. THE MAGIC TRICK: "Sliding Window" of 2 lines
        # We combine Line 1+2, then Line 2+3, then Line 3+4...
        for i in range(len(clean_lines) - 1):
            # Combine current line and next line
            combined_segment = f"{clean_lines[i]} {clean_lines[i+1]}"

            # Only keep if it's long enough (e.g. > 40 chars)
            if len(combined_segment) > 40:
                corpus.append(combined_segment)

print(f"✅ Processing complete. Total segments: {len(corpus)}")
# Now 'corpus' is ready for SBERT

🧹 Cleaning and Segmenting Lyrics...
✅ Processing complete. Total segments: 12767


## Filter/replace words

In [29]:
# Download resources (run once)
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')

def get_top_keywords_filtered(texts, n=10):
    # 1. SETUP LISTS
    custom_stops = list(nltk.corpus.stopwords.words('english'))
    pop_slang = [
        "lil", "gon", "bout", "em", "ayy", "uh", "huh", "ha", "vi", "doo", "wee",
        "yeah", "oh", "baby", "know", "like", "got", "just", "don", "ve", "ll", 
        "want", "need", "love", "time", "way", "make", "say", "come", "go", "right",
        "look", "good", "feel", "really", "cause", "wanna", "gonna", "gotta", "ain",
        "girl", "boy", "man", "woman", "hey", "ooh", "whoa", "shit", "fuck", "bitch",
        "nigga", "niggas", "damn", "ass", "tell", "think", "never", "back", "let",
        "swag", "yuh", "hum", "who", "what", "where", "why", "top", "call", "put",
        "gang", "thug", "bro", "pussy", "tryna", "chick", "girls", "slatt", "mmh"
    ]
    
    abstract_concepts = [
        "life", "day", "night", "heart", "mind", "world", "everything", "nothing", 
        "things", "nothin", "songs", "song", "name", "eyes", "face", "voice", 
        "head", "hand", "hands", "god", "soul", "mind", "pain", "hope", "wish", 
        "fame", "lie", "lies", "truth", "word", "words", "end", "reason", "part",
        "told", "saw", "knew", "met", "said", "made", "found", "came", "went",
        "die", "born", "live", "dead", "death", "control", "move", "wait", "hold",
        "stop", "start", "change", "keep", "leave", "stay", "believe", "remember"
    ]
    
    NAME_REPLACEMENTS = {
        "regina": "queen", "veronica": "girl", "heather": "student",
        "hamilton": "soldier", "burr": "man", "jefferson": "politician",
        "elphaba": "witch", "glinda": "friend", "evan": "boy",
        "connor": "friend", "usnavi": "guy", "vanessa": "girl"
    }

    # 2. APPLY NAME REPLACEMENTS (The Fix)
    processed_texts = []
    for t in texts:
        t_lower = t.lower() # Lowercase first
        for name, replacement in NAME_REPLACEMENTS.items():
            t_lower = t_lower.replace(name, replacement)
        processed_texts.append(t_lower)

    # 3. VECTORIZE (Use the processed list!)
    all_stops = custom_stops + pop_slang + abstract_concepts
    
    try:
        vec = CountVectorizer(stop_words=all_stops).fit(processed_texts)
        bag_of_words = vec.transform(processed_texts)
        sum_words = bag_of_words.sum(axis=0) 
        words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
        
        # 4. FINAL CLEANUP (Noun Check)
        cleaned_list = []
        for word, freq in sorted(words_freq, key=lambda x: x[1], reverse=True):
            if len(word) <= 2: continue 
            
            # Check Noun (NN, NNS)
            # We wrap it in list [] because pos_tag expects a list of tokens
            pos_tag = nltk.pos_tag([word])[0][1]
            if pos_tag.startswith('NN'):
                cleaned_list.append((word, freq))
                
        return cleaned_list[:n]
        
    except ValueError:
        # Handles empty clusters
        return []

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Bay_Techatham/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/Bay_Techatham/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Bay_Techatham/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
import optuna
from sentence_transformers import SentenceTransformer

# 0. ENSURE EMBEDDINGS EXIST
if 'embeddings' not in globals():
    print("⏳ Generating Embeddings (this happens once)...")
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(corpus)


# 1. DEFINE THE OBJECTIVE FUNCTION
# Optuna will run this function many times with different 'k' values
def objective(trial):
    k = trial.suggest_int('n_clusters', 40, 60)

    model = KMeans(
        n_clusters=k,
        random_state=42,
        n_init=10
    )
    labels = model.fit_predict(embeddings)

    cohesion = cluster_cohesion(embeddings, labels)

    return cohesion  # HIGHER = better semantic cohesion

# 2. RUN THE OPTIMIZATION
print("🤖 Starting Optuna Optimization...")
study = optuna.create_study(direction='maximize')  # We want MAX separation
study.optimize(objective, n_trials=20)  # Try 20 different times

# 3. PRINT RESULTS
print("-" * 40)
print(f"🏆 Best K found: {study.best_params['n_clusters']}")
print(f"📈 Best Silhouette Score: {study.best_value:.4f}")
print("-" * 40)

# 4. OPTIONAL: VISUALIZE
optuna.visualization.plot_optimization_history(study).show()

[I 2026-01-12 02:15:20,096] A new study created in memory with name: no-name-39618001-509f-482e-8413-a21059baa870


🤖 Starting Optuna Optimization...


[I 2026-01-12 02:15:37,715] Trial 0 finished with value: 0.2716422975063324 and parameters: {'n_clusters': 48}. Best is trial 0 with value: 0.2716422975063324.
[I 2026-01-12 02:15:52,311] Trial 1 finished with value: 0.2559882402420044 and parameters: {'n_clusters': 41}. Best is trial 0 with value: 0.2716422975063324.
[I 2026-01-12 02:16:12,704] Trial 2 finished with value: 0.27651482820510864 and parameters: {'n_clusters': 59}. Best is trial 2 with value: 0.27651482820510864.
[I 2026-01-12 02:16:31,916] Trial 3 finished with value: 0.27528074383735657 and parameters: {'n_clusters': 60}. Best is trial 2 with value: 0.27651482820510864.
[I 2026-01-12 02:16:51,851] Trial 4 finished with value: 0.27528074383735657 and parameters: {'n_clusters': 60}. Best is trial 2 with value: 0.27651482820510864.
[I 2026-01-12 02:17:09,412] Trial 5 finished with value: 0.266170859336853 and parameters: {'n_clusters': 50}. Best is trial 2 with value: 0.27651482820510864.
[I 2026-01-12 02:17:26,276] Trial 

----------------------------------------
🏆 Best K found: 59
📈 Best Silhouette Score: 0.2765
----------------------------------------


In [38]:
print(f"\n🧠 Analyzing {len(corpus)} lyric segments with SBERT...")

NUM_CLUSTERS = 10

# A. Embed
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(corpus)

# B. Cluster
kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42)
kmeans.fit(embeddings)
cluster_labels = kmeans.labels_

# ==========================================
# 4. RESULTS (THE "SHOPPING LIST")
# ==========================================
print("\n" + "="*40)
print("   YOUR VISUAL SHOPPING LIST")
print("="*40)

# Helper to find words from the clusters
def get_top_keywords(texts, n=8):
    vec = CountVectorizer(stop_words='english').fit(texts)
    bag_of_words = vec.transform(texts)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    return sorted(words_freq, key=lambda x: x[1], reverse=True)[:n]

df = pd.DataFrame({'text': corpus, 'cluster': cluster_labels})

for i in range(NUM_CLUSTERS):
    cluster_text = df[df['cluster'] == i]['text'].tolist()
    
    # Use the new function
    keywords = get_top_keywords_filtered(cluster_text)
    
    print(f"\n📁 CLUSTER {i+1} (Sample: {len(cluster_text)})")
    # Print just the words, not the frequency counts
    print(f"   Keywords: {[k[0] for k in keywords]}")
    print(f"   👉 DECISION: What 3D asset matches these nouns?")


🧠 Analyzing 12767 lyric segments with SBERT...


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 3d3d4059-536c-4490-b8b5-1534b093369d)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].



   YOUR VISUAL SHOPPING LIST

📁 CLUSTER 1 (Sample: 1294)
   Keywords: ['woah', 'shine', 'shake', 'hear', 'sound', 'dat', 'shot', 'dang', 'fight', 'wolf']
   👉 DECISION: What 3D asset matches these nouns?

📁 CLUSTER 2 (Sample: 1503)
   Keywords: ['mine', 'belong', 'home', 'sorry', 'friend', 'beautiful', 'son', 'show', 'shot', 'years']
   👉 DECISION: What 3D asset matches these nouns?

📁 CLUSTER 3 (Sample: 1355)
   Keywords: ['fight', 'men', 'war', 'captain', 'people', 'deserves', 'lives', 'mercy', 'win', 'home']
   👉 DECISION: What 3D asset matches these nouns?

📁 CLUSTER 4 (Sample: 1678)
   Keywords: ['try', 'game', 'thing', 'fight', 'til', 'something', 'line', 'break', 'monster', 'work']
   👉 DECISION: What 3D asset matches these nouns?

📁 CLUSTER 5 (Sample: 634)
   Keywords: ['soldier', 'president', 'alexander', 'politician', 'washington', 'sir', 'york', 'thomas', 'home', 'congress']
   👉 DECISION: What 3D asset matches these nouns?

📁 CLUSTER 6 (Sample: 1312)
   Keywords: ['room', 

In [24]:
import pickle

# 1. SAVE THE MODEL
# (Make sure 'kmeans' is your trained model variable)
# Using joblib is often safer for sklearn models than pickle, but pickle works too.
with open('kmeans_presets.pkl', 'wb') as f:
    pickle.dump(kmeans, f)
print("✅ Model saved as 'kmeans_presets.pkl'")

# 2. SAVE YOUR ASSET MAP
# Mapped to match your SBERT Keywords exactly:

preset_map = {
    0: "urban_environment",  # Cluster 1: Money, Piragua, Bag (In The Heights/City)
    1: "urban_environment",  # Cluster 2: Dance, Shake, Shot (Party/Club Vibe)
    2: "urban_environment",  # Cluster 3: Heather, Regina, Queen (High School/Social)
    3: "room_environment",   # Cluster 4: Home, Door, Walk (Domestic/Inside)
    4: "sky_environment",    # Cluster 5: Sun, Light, Sky, Stars (Lanterns/Magic)
    5: "dark_environment",   # Cluster 6: Hate, Fight, Care (Conflict/Spooky)
    6: "ocean_environment",  # Cluster 7: Captain, War, Men, Stand (Epic/Sea)
    7: "fire_environment",   # Cluster 8: Hamilton, Fire, Burn (Revolution/Rage)
    8: "snow_environment",   # Cluster 9: Christmas, Merry, Fall (Winter)
    9: "room_environment"    # Cluster 10: Room, Dream, Sleep (Intimate/Thoughts)
}

with open('preset_labels.pkl', 'wb') as f:
    pickle.dump(preset_map, f)
print("✅ Labels saved as 'preset_labels.pkl'")

✅ Model saved as 'kmeans_presets.pkl'
✅ Labels saved as 'preset_labels.pkl'


In [50]:
# 1. CORRECT WAY TO LOAD
with open('output/pickle/kmeans_presets(musical_and_pop).pkl', 'rb') as f:
    kk = pickle.load(f)

with open('output/pickle/preset_labels(musical_and_pop).pkl', 'rb') as f:
    label_map = pickle.load(f)
    
# Check the difference between your current text and your saved brain
print(f"📉 Current Corpus Length: {len(corpus)}")
print(f"🧠 Saved Model Labels:    {len(kk.labels_)}")

# 2. RUN YOUR ANALYSIS
print("\n" + "="*40)
print("   YOUR VISUAL SHOPPING LIST")
print("="*40)

# ⚠️ CRITICAL NOTE: 'corpus' must exist in memory!
# If this is a new script, you need to re-generate or load the 'corpus' list
# exactly as it was during training, otherwise model.labels_ won't match.
if 'corpus' not in locals():
    print("❌ ERROR: Variable 'corpus' is missing. You need the text data to analyze keywords.")
else:
    df = pd.DataFrame({'text': corpus, 'cluster': kk.labels_})

    # Use NUM_CLUSTERS from the loaded model to be safe
    num_clusters = kk.n_clusters

    for i in range(num_clusters):
        cluster_text = df[df['cluster'] == i]['text'].tolist()
        
        # Use your custom filter function
        keywords = get_top_keywords_filtered(cluster_text, 100)
        
        # Get the label you saved (if available) to verify
        saved_label = label_map.get(i, "Unknown")
        
        print(f"\n📁 CLUSTER {i} (Sample: {len(cluster_text)})")
        print(f"   🏷️ Saved Label: {saved_label}") 
        print(f"   Keywords: {[k[0] for k in keywords]}")

📉 Current Corpus Length: 12767
🧠 Saved Model Labels:    12767

   YOUR VISUAL SHOPPING LIST

📁 CLUSTER 0 (Sample: 1294)
   🏷️ Saved Label: ('urban_environment', 1.0)
   Keywords: ['woah', 'shine', 'shake', 'hear', 'sound', 'dat', 'shot', 'dang', 'fight', 'wolf', 'home', 'work', 'sigue', 'knock', 'beautiful', 'light', 'sung', 'rise', 'alabanza', 'senora', 'throwin', 'goodbye', 'nahmen', 'heard', 'turn', 'strike', 'piragua', 'lai', 'dance', 'beep', 'voices', 'room', 'guy', 'eleka', 'count', 'watch', 'drownin', 'break', 'til', 'thank', 'write', 'pare', 'bandera', 'tum', 'boo', 'teach', 'boom', 'sept', 'huit', 'neuf', 'daylight', 'respira', 'calor', 'yes', 'blue', 'something', 'tune', 'happens', 'bop', 'meet', 'cinq', 'qué', 'ride', 'heia', 'squi', 'deux', 'trois', 'quatre', 'diggety', 'westerberg', 'odysseus', 'bite', 'mis', 'everyone', 'people', 'melody', 'play', 'mean', 'kind', 'mine', 'hello', 'balls', 'whine', 'esa', 'powerless', 'try', 'music', 'king', 'beat', 'forgotten', 'helpless'

In [42]:
preset_map = {
    0: ("urban_environment", 1.0),
    1: ("urban_environment", 0.8),
    2: ("urban_environment", 0.9),
    3: ("room_environment", 1.0),
    4: ("sky_environment", 1.0),
    5: ("dark_environment", 1.0),
    6: ("ocean_environment", 1.0),
    7: ("fire_environment", 1.0),
    8: ("snow_environment", 1.0),
    9: ("room_environment", 0.9)
}

with open('output/pickle/preset_labels(musical_and_pop).pkl', 'wb') as f:
    pickle.dump(preset_map, f)

In [47]:
musical_data.to_csv("dataset/musical.csv", index=False)

print("File restored successfully!")

File restored successfully!
